# Titanic Kaggle Competetion:
Kaggle Score : 0.80382<br/>
Predict survival on the Titanic

In [ ]:
# import all the required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix,roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier
#from google.colab import files
from xgboost import XGBClassifier
from xgboost import plot_importance

sns.set()

# Peeking into the dataset:
To start with we will take a peek into the dataset.

In [ ]:
#Load the training and testing data 
url_train = "../input/train.csv"
url_test = "../input/test.csv"

df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)


In [ ]:
# take a look in to head to see if the data has been loaded correctly.
df_train.head(3)

In [ ]:
df_train.info()

Columns having null values or missing entries: Age, Cabin and Embarked. <br/>
The cabin column has only 204 observations(23%). Since a large no. of observations are missing, we kight be templted to drop this column, but later we will see how this column can be used as to derive a new feaure.
Later, we will have to device a way to impute the values of the Age column which has close to 20% missing values.<br/>
Embarked has only values missing, we can either impute those using mode or some other startegies. Deleting also wont make much of a difference


In [ ]:
df_train.describe()

From the above description it is clear that Age and Fare is are normaly distributed. Where as Parch and SibSp have Left Skew.

# It is very important to understand the columns of the dataset.
### Below is the description of each of the columns present in the dataset.
* PassengerId: Unique id of each passenger
* Survived: Survival status of the passenger. (1 = Survived and 0 = Didn't Survive)
* Pclass: Class in which the passenger. (1 = 1st class, 2 = 2nd class, 3 = 3rd class)
* Name: Name of the passenger
* Sex: Gender of the passenger 
* Age: Age of the passenger in years
* SibSp: Number of sibling spouse on board
* Parch: Number of parent children on board
* Ticket: Ticket No. of the passenger
* Fare: Ticket fare of the passenger in British Pound
* Cabin: Cabin of the passeneger
* Embarked: Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

# Exploratory Data Analysis:
From the below EDA, it is clear that following features have impact on the survival:
1. Pclass
2. Sex
3. Age_grp ( Derived feature from Age)
4. Embarked
5. Whether the passenger has cabin or not (Derived Feature from Cabin)
6. Family Size (Derived Features from Parch and SibSp)



### Univariate Analysis:
Finding:<br/>
Qualitative/Categorical Columns:
1. Pclass
2. Sex
3. Embarked

Quantitative/Continous Columns:
1. Age
2. Parch
3. SibSp
4. Fare


#### 1. Survival
---
Observatoin:
* Base rate of Survival:  0.38
* Base rate of Non Survival:  0.62


In [ ]:
survival = pd.DataFrame(df_train.Survived.value_counts())
survival["ratio"] = survival.Survived/survival.Survived.sum()
survival.sort_index(inplace=True)
survival.index=["Not Survived","Survived"]
survival.reset_index(inplace=True)
survival.columns =["status","count","ratio"]


survival

In [ ]:
def plot_pie_chart(data,title,label_col):
    color_list = np.array(['r','g', 'b','c','y'])
    plt.pie(data["count"],colors=color_list[:data.shape[0]],autopct='%.2f')
    plt.axis('equal')
    plt.title(title)
    plt.legend(data[label_col])
    plt.show()

In [ ]:
plot_pie_chart(survival,"Survived vs Not Survived",'status')

observation : <br/>
* 62% of people onboard couldn't survive 

#### 2. Passenger class
---

In [ ]:
passenger_class = pd.DataFrame(df_train.Pclass.value_counts())
passenger_class["ratio"] = passenger_class.Pclass/passenger_class.Pclass.sum()
passenger_class.columns = ['count','ratio']
passenger_class.sort_index(ascending=True,inplace=True)
#passenger_class.index=['1st','2nd','3rd']
passenger_class.reset_index(inplace=True)
passenger_class.columns = ['Pclass','count','ratio']
passenger_class

In [ ]:
plot_pie_chart(passenger_class,'Classwise Breakup of Passengers','Pclass')

##### observation: 
* Most of the people(55%) were travelling in 3rd class. 


#### 3.Gender 


Overall passengers:

In [ ]:
passenger_gender = pd.DataFrame(df_train.Sex.value_counts())
passenger_gender["ratio"] = passenger_gender.Sex/passenger_gender.Sex.sum()
passenger_gender.sort_index(ascending=True)
passenger_gender.columns=['count','ratio']
passenger_gender.reset_index(inplace=True)
passenger_gender.columns=['Sex','count','ratio']
passenger_gender

In [ ]:
plot_pie_chart(passenger_gender,"Genderwise Breakup",'Sex')

#### 4. Age

Age has missing values.
Before doing any anaysis on the age we will have to impute the missing values. <br/>

Simplest would be to calculate mean, but we will try and use conditional mean.<br/>
We will find the saluation of each passeneger and assign the age as per the saluation mean


In [ ]:
print ("Observations with missing age:",df_train.Age.isna().sum())

In [ ]:
sample_name="Conlon, Mr. Thomas Henry"
def get_salutation(full_name):
    '''
    This method takes in the name and spits out the caption from the name
    
    '''
    start = str(full_name).find(',')
    end= str(full_name).find('.')
    salutation = full_name[start+1:end]
    return salutation



In [ ]:
df_train["salutation"] = df_train.Name.apply(get_salutation)
df_train.salutation = df_train.salutation.str.strip()
df_train.salutation = df_train.salutation.str.lower()

In [ ]:
mean_ages = df_train.groupby(["salutation","Pclass"]).agg({"Age":"mean"}).reset_index()
age_unknown = df_train[df_train.Age.isna()]
imputed_age = pd.merge(age_unknown,mean_ages,on=['salutation','Pclass'],how='left')[["PassengerId","Age_y"]]
data = pd.merge(df_train,imputed_age,on=["PassengerId"], how='left')
data.Age.fillna(0,inplace=True)
data.Age_y.fillna(0,inplace=True)
data.Age = data.Age+data.Age_y
df_train =data
df_train.Age = df_train.Age.round()

In [ ]:
df_train.Age.describe()

In [ ]:
df_train.Age.plot(kind='kde',color='c')
plt.xlabel("Age")
plt.title("Passenger Age Distribution")
plt.show()

In [ ]:
plt.hist(df_train.Age,cumulative=1,density=True,histtype='step',color='c')
plt.show()

#### 5. Tickets:

In [ ]:
print ("Unique passengers:",len(df_train.PassengerId.unique()))
print ("Unique tickets:",len(df_train.Ticket.unique()))


This is because passengers travelling togehther have the same Ticket Number

#### 6. Ticket Fare

In [ ]:
plt.title("Fare histogram")
df_train.Fare.hist(color='c')
plt.xlabel("Fare")
plt.ylabel("Frequency")
plt.xticks(np.arange(0,500,50))
plt.show()

In [ ]:
df_train.Fare.plot(kind='kde',color='c')
plt.show()

In [ ]:
df_train.Fare.describe()

In [ ]:
print ("passengers with zero fare: ",(df_train.Fare == 0).sum())

#### 7. Parch

In [ ]:
df_train["Parch"].describe()

In [ ]:
df_train["Parch"].plot(kind='kde',color='c')
plt.title("Parch Distribution")
plt.xlabel("Parent/Children")
plt.show()

#### 8. Sibsp

In [ ]:
df_train["SibSp"].describe()

In [ ]:
df_train["SibSp"].plot(kind='kde',color='c')
plt.title("SibSp Distribution")
plt.xlabel("Sibling/Spouse")
plt.show()

#### Family Size

Let's create a new column family size by combining SibSp and Parch

In [ ]:
df_train["family_size"]=df_train["SibSp"]+df_train["Parch"]

In [ ]:
df_train.family_size.describe()

In [ ]:
df_train.family_size.plot(kind='kde')
plt.xlabel("Family Size")
plt.show()

In [ ]:
survivors = df_train[df_train["Survived"]==1]
non_survivors = df_train[df_train["Survived"]==0]
survivors.family_size.plot(kind='kde',color='g')
non_survivors.family_size.plot(kind='kde',color='r')
plt.legend(["Survivor","Non-survivor"])
plt.xlabel("family_size")
plt.title("Passenger family_size Distribution")
plt.xticks(np.arange(0,10,1))
plt.show()

#### 9. Embarked:

This column contains 2-3 null values.
They can be either ignored or imputed as mode

In [ ]:
#impute the missing values for Embarked as mode of the column
df_train.loc[df_train["Embarked"].isna(),"Embarked"] = 'S'

In [ ]:
#Assert if null values have been removed
assert df_train["Embarked"].isna().sum() == 0

In [ ]:
embarked_data =pd.DataFrame(df_train["Embarked"].value_counts())
embarked_data["ratio"] = embarked_data.Embarked/embarked_data.Embarked.sum()

embarked_data.reset_index(inplace=True)
embarked_data.columns=['Embarked','count','ratio']
embarked_data


In [ ]:
plot_pie_chart(embarked_data,"Port of Embarkation","Embarked")

#### 10. Cabin

In [ ]:
(float(df_train["Cabin"].isna().sum())/df_train["Cabin"].shape[0])*100

In [ ]:
df_train["has_cabin"] = 1
df_train.loc[df_train["Cabin"].isna(),"has_cabin" ] = 0

In [ ]:
cabin = pd.DataFrame(df_train.has_cabin.value_counts()).reset_index()
cabin.columns=["cabin","count"]
cabin.loc[cabin.cabin == 0 , "cabin"] = "don't have"
cabin.loc[cabin.cabin == 1 , "cabin"] = "have" 


In [ ]:
plot_pie_chart(cabin,"Cabinwise Passenger","cabin")

### Bivariate Analysis:
Summary:


#### Passenger Class and Survived:
Observation:
* There is large variation between the survivals of passengers of different class
* This can act as a good predictor

In [ ]:
pass_class_survival = df_train.groupby(['Pclass','Survived']).agg({"PassengerId":"count"})
pass_class_survival.reset_index(inplace=True)
pass_class_survival = pd.merge(pass_class_survival,passenger_class,on=["Pclass"],how='left')
pass_class_survival["ratio"] = pass_class_survival["PassengerId"]/pass_class_survival["count"]
pass_class_survival= pass_class_survival[pass_class_survival.Survived==1][['Pclass','ratio']]
pass_class_survival.columns=['Pclass','prob_survival']
pass_class_survival

#### Passenger Class and Fare

In [ ]:
df_train[["Pclass","Fare"]].corr()

#### Gender and Survived
Observation:
* The gender is playing a very important role 
* This again is a good candidate for feature

In [ ]:
gender_survival = df_train.groupby(["Sex","Survived"]).agg({"PassengerId":"count"})
gender_survival.reset_index(inplace=True)
gender_survival = pd.merge(gender_survival,passenger_gender,on=['Sex'], how ='left')
gender_survival['prob_survival']=gender_survival['PassengerId']/gender_survival['count']
gender_survival = gender_survival[gender_survival.Survived==1][["Sex","prob_survival"]]
gender_survival

#### Age and Survived:
Since *Age* is a continous variable. Let's check the correlation with Survived

Observation:
* As such there is no strong correaltion between Age and Survival.
* If age is broken down into groups like child, teenager and adult, we see that probability of survival goes on decreasing in the matured group.

In [ ]:
print("Correaltion between Age and Survival")
df_train[["Age","Survived"]].corr()

In [ ]:
sns.boxplot(y='Age',x='Survived',data=df_train)
plt.title("Age and Survival")
plt.plot()

In [ ]:
survivors = df_train[df_train["Survived"]==1]
non_survivors = df_train[df_train["Survived"]==0]
survivors.Age.plot(kind='kde',color='g')
non_survivors.Age.plot(kind='kde',color='r')
plt.legend(["Survivor","Non-survivor"])
plt.xlabel("Age")
plt.title("Survivors vs Non Survivors Age Distribution")
plt.show()

We see that:<br/>
* Between the age 0-15 : More passengers have survived than dead*
* Between the age 15-35 : More passengers have died than survived
* After 35 : More or less the same

Using the above observation we create a new categorical feature 'Age_grp' with values:<br/>
* child : for value between 0 and 18
* adults : for value between 18 and 35
* senior: for values greather than 35

In [ ]:

def get_age_group(age):
   if(age<=18):
      return 'child'
   elif((age>18)& (age<=35)):
     return 'adults'
   elif((age>35)):
         return 'senior'
#    elif((age>45) &(age<=60)):
#          return '45-60'
#    elif((age>60)):
#       return '60+'
        
      #ge_labels = ['15-','15-35','35-45','40-60','60+']


In [ ]:
df_train["Age_grp"] = df_train.Age.apply(get_age_group)

In [ ]:
df_train["Age_grp"].value_counts()

In [ ]:
age_grp_survival = df_train.groupby(["Age_grp","Survived"]).agg({"PassengerId":"count"}).reset_index()
age_grp_passengers = df_train.groupby(["Age_grp"]).agg({"PassengerId":"count"}).reset_index()

age_grp_survival = pd.merge(age_grp_survival,age_grp_passengers,on=["Age_grp"], how="left")
age_grp_survival["prob_survival"] = age_grp_survival["PassengerId_x"]/age_grp_survival["PassengerId_y"]
age_grp_survival[age_grp_survival.Survived == 1][["Age_grp","prob_survival"]]

In [ ]:
#### Fare and Survived

In [ ]:
survivors = df_train[df_train["Survived"]==1]
non_survivors = df_train[df_train["Survived"]==0]
survivors.Fare.plot(kind='kde',color='g')
non_survivors.Fare.plot(kind='kde',color='r')
plt.legend(["Survivor","Non-survivor"])
plt.xlabel("Fare")
plt.title("Passenger Fare Distribution")
plt.xlim(0,200)
plt.xticks(np.arange(0,200,10))
plt.show()

We see that:<br/>
* Between the fare 0-30 : More passengers have died than survived
* Between the age 30-100: More passengers have survived than died
* Between the age 100-150: More passengers have survived than died, but the survivors are lower than the previous group
* After 150: The survived and died passengers are kind of same 

Using the above observation we create a new categorical feature 'Fare_grp' with values:<br/>
* Between the fare 0-30 : 0-30
* Between the age 30-100: 30-100
* Between the age 100-150: -100-150
* After 150: 150+

#### Fare and Class:



In [ ]:
sns.boxplot(y='Fare',x='Pclass',data=df_train)
plt.title("Class and Fare")
plt.plot()

In [ ]:
df_train.groupby(["Pclass"]).agg({"Fare":"mean"})

As expected the Fare increases as you move to the Higher classes.<br/>
The fare differrence isnt very large between 2nd and 3rd class, but very large between 2nd and 1st class`


In [ ]:
def group_fare(fare):
  if fare <= 30:
    return "0-30"
  elif fare >30 and fare<=100:
    return "30-100"
  elif fare >100 and fare < 150:
    return "100-150"
  elif fare > 150:
    return "150+"

In [ ]:
df_train["fare_group"] = df_train["Fare"].apply(group_fare)



#### Parch and Survived

Observation:
* There is no corealtion between Parch and Survived


In [ ]:
df_train[["Survived","Parch"]].corr()

#### SibSp and Survived:
Observation:
* There is no corealtion between Parch and Survived


In [ ]:
df_train[["Survived","SibSp"]].corr()

#### Family Size and Survived:

In [ ]:
df_train["family_size"] = df_train["SibSp"]+df_train["Parch"]

In [ ]:
print ("Distribution of Family Size:")
print (df_train["family_size"].describe())

In [ ]:
df_train[["family_size","Survived"]].corr()

In [ ]:
df_train[df_train["Survived"]==1]["family_size"].plot(kind='kde',color='g')
df_train[df_train["Survived"]==0]["family_size"].plot(kind='kde',color='r')
plt.xticks(np.arange(0,15,1))
plt.show()

From the plot, it is visible that:
* Between 1-3 : The no. of survivors are greater than non survivors
* Between 4 -6 : The no. of non survivors are greater than survivors
* After 6: It is pretty much the same

In [ ]:
def group_family_size(size):
  if size == 0:
    return "single"
  elif size >0 and size<=3:
    return "small"
  elif size >3 :
    return "medium"
 

In [ ]:
df_train["family"] = df_train["SibSp"]+df_train["Parch"]

df_train["family_size"] = df_train["family"].apply(group_family_size)

#### Embarked and Survival
Observation:
* Prob of survival varies significantly depending on Port of Embarkation
* This is a strong feature candidate

In [ ]:
port_survival = df_train.groupby(["Embarked","Survived"]).agg({"PassengerId":"count"})
port_survival.reset_index(inplace=True)
port_survival = pd.merge(port_survival,embarked_data,on=["Embarked"],how="left")
port_survival.columns=["Embarked","Survived","count","total_count","prob_survival"]
port_survival["prob_survival"] = port_survival["count"]/port_survival["total_count"]
port_survival[port_survival["Survived"] == 1][["Embarked","prob_survival"]]

### Multivariate Analysis

#### All Columns

In [ ]:

sns.heatmap(df_train.corr())
plt.show()

#### Class, Gender and Survived:
Observation:
* The probability of survival for females in 1st and 2nd class is very 90%. 
* Ladies in the 3rd class have it as 50%
* Males have the best chance of survival in First class(36%). It dips to 15% and 13% in the lower classes.


In [ ]:
class_sex_survivor_data = df_train.groupby(["Pclass","Sex","Survived"]).agg({"PassengerId":"count"}).reset_index()
class_sex_data = class_sex_survivor_data.groupby(["Pclass","Sex"]).agg({"PassengerId":"sum"}).reset_index()
class_sex_survivor_data = pd.merge(class_sex_survivor_data,class_sex_data,on=["Pclass","Sex"],how="left")
class_sex_survivor_data["prob_survival"] = class_sex_survivor_data["PassengerId_x"]/class_sex_survivor_data["PassengerId_y"]
survivors_class_gender = class_sex_survivor_data[class_sex_survivor_data["Survived"]==1][["Pclass","Sex","prob_survival"]]

In [ ]:
male_data = survivors_class_gender[survivors_class_gender["Sex"] == "male"]["prob_survival"]
female_data = survivors_class_gender[survivors_class_gender["Sex"] == "female"]["prob_survival"]

ind = np.arange(3) 
width = 0.35       
plt.bar(ind, male_data, width, label='Men')
plt.bar(ind + width, female_data, width,label='Women')

plt.xlabel("Class")
plt.ylabel('Prob of Survival')
plt.title('Survival Probability by Class and Gender')

plt.xticks(ind + width / 2, ('1', '2', '3'))
plt.legend(loc='best')
plt.show()

# Predictive Modeling using Gradient Boosting
Predicting Survivors using XGB Boost

### Preprocessing of Testing Data

#### Imputation of missing age

In [ ]:

df_test["salutation"] = df_test.Name.apply(get_salutation)
df_test.salutation = df_test.salutation.str.strip()
df_test.salutation = df_test.salutation.str.lower()

In [ ]:
mean_ages = df_test.groupby(["salutation","Pclass"]).agg({"Age":"mean"}).reset_index()
age_unknown = df_test[df_test.Age.isna()]
imputed_age = pd.merge(age_unknown,mean_ages,on=['salutation','Pclass'],how='left')[["PassengerId","Age_y"]]
data = pd.merge(df_test,imputed_age,on=["PassengerId"], how='left')
data.Age.fillna(0,inplace=True)
data.Age_y.fillna(0,inplace=True)
data.Age = data.Age+data.Age_y
df_test =data
df_test.Age = df_test.Age.round()

In [ ]:
df_test["Age"].describe()

In [ ]:
df_test["Age_grp"] = df_test.Age.apply(get_age_group)

In [ ]:
df_test["family"] = df_test["SibSp"]+df_test["Parch"]
df_test["family_size"] = df_test["family"].apply(group_family_size)
df_test["fare_group"] = df_test["Fare"].apply(group_fare)
    
  

## XGBoost

### first quick implemenation
Kaggle Score: 0.78947

In [ ]:
def get_predictor_target_data(data,features,target):
    predictor_variables=features
    target_variable=target
    df_training_data = data[predictor_variables]
    try:
      df_target_data = data[target_variable]
    except:
      df_target_data=None
    
    return df_training_data, df_target_data


In [ ]:
def get_sub_mission_files():
    df_submission= pd.DataFrame(df_test["PassengerId"])
    df_submission["Survived"]=predictions
    df_submission.to_csv("submission.csv",index=False)
    print("submissions saved")
    #files.download("submission.csv")
    



In [ ]:
def filter_cols(colname,filters):
  if colname.startswith('salutation'):
    if colname.endswith(tuple(np.intersect1d(np.array(test_OHE.columns),np.array(train_data_OHE.columns)))):
      return True
    else:
      return False
  elif(colname in filters):
    return False
  else:
        return True

In [ ]:
df_test.head(3)

In [ ]:
# Kaggle Score : 0.80382.
#shuffle the dataframe
feature_filter=['Age_grp_child','salutation_miss','Age_grp_adults','fare_group_100-150','family_size_single','Embarked_C']
df_train["family"] =df_train["Parch"]+df_train["SibSp"]
df_test["family"] =df_test["Parch"]+df_test["SibSp"]

df_train["has_cabin"] = 1
df_train.loc[df_train["Cabin"].isna(),"has_cabin" ] = 0

df_test["has_cabin"] = 1
df_test.loc[df_test["Cabin"].isna(),"has_cabin" ] = 0

#Features:
# feature_filter=[]
# features=['PassengerId','Pclass','Sex','Embarked','Age_grp']
# cat_features=['Pclass','Sex','Embarked','Age_grp']
# target_var=['Survived']
features=['PassengerId','Pclass','Sex','Embarked','Age_grp',"family_size",'salutation',"has_cabin"]
cat_features=['Pclass','Sex','Embarked','Age_grp',"family_size",'salutation']
target_var=['Survived']

#Shuffle the data
df_train = df_train.sample(frac=1,random_state=42)

#Separate dependent and independant variables of training data:
train_data,target_data = get_predictor_target_data(df_train,features,target_var)


#Perform One Hot Encoding
train_data_OHE = pd.get_dummies(train_data,columns=cat_features,drop_first=True)

#Separate dependent and independant variables of final testing data:
test_predictor,test_target = get_predictor_target_data(df_test,features,target_var)

#Perform One Hot Encoding
test_OHE = pd.get_dummies(test_predictor,columns=cat_features,drop_first=True)

#Filter unwanted columns
train_data_OHE = train_data_OHE[[x for x in train_data_OHE.columns if filter_cols(x,feature_filter)]]

#Convert to numpy array
arr_training_data = train_data_OHE.values
arr_target_data = target_data.values.reshape(-1)


train_X, test_X,train_y,test_y = train_test_split(arr_training_data,arr_target_data,test_size=0.4,random_state=42, stratify=arr_target_data)


cv_X,test_X,cv_y,test_y = train_test_split(test_X,test_y,test_size=0.2,random_state=42, stratify=test_y)

eval_set = [(cv_X[:,1:], cv_y)]
#intialise the model
xgb_model = XGBClassifier(eta=0.01,objective="binary:logistic",random_state=42)

#fit it
xgb_model.fit(train_X[:,1:], train_y, early_stopping_rounds = 50, eval_metric = "error", eval_set = eval_set, verbose = False)

#predict on training set
training_predictions = xgb_model.predict(train_X[:,1:])
#predict on cross validation set
cv_predictions = xgb_model.predict(cv_X[:,1:])
#predict on testing set
test_predictions = xgb_model.predict(test_X[:,1:])



print("Training Metrics:")
print(confusion_matrix(train_y,training_predictions))
print(classification_report(train_y,training_predictions))
print("Train accuracy:",accuracy_score(train_y,training_predictions))
print("Train auc score:",roc_auc_score(train_y,training_predictions))

print("CV Metrics:")
#prnt confusion matrix and classification report
print(confusion_matrix(cv_y,cv_predictions))
print(classification_report(cv_y,cv_predictions))
print("CV accuracy:",accuracy_score(cv_y,cv_predictions))
print("CV auc score:",roc_auc_score(cv_y,cv_predictions))

print("Testing Metrics:")
#prnt confusion matrix and classification report
print(confusion_matrix(test_y,test_predictions))
print(classification_report(test_y,test_predictions))
print("Test accuracy:",accuracy_score(test_y,test_predictions))
print("Test auc score:",roc_auc_score(test_y,test_predictions))



print("Creating Submission...")
test_OHE = test_OHE[[x for x in test_OHE.columns if filter_cols(x,feature_filter)]]
predictions = xgb_model.predict(test_OHE.drop(['PassengerId'],axis=1).values)
# print(confusion_matrix(test_target.astype(int),predictions))
# print(classification_report(test_target.astype(int),predictions))
# print("Test accuracy:",accuracy_score(test_target.astype(int),predictions))
# print("Test auc score:",roc_auc_score(test_target.astype(int),predictions))


get_sub_mission_files()


## Plot important features

In [ ]:

#plt.bar(range(len(xgb_model.feature_importances_)), xgb_model.feature_importances_)
plt.bar( train_data_OHE.columns[1:],xgb_model.feature_importances_)
plt.xticks(train_data_OHE.columns[1:] ,rotation=90)
plt.show()